In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import librosa
import torch
import torchaudio
from SED_CRNN import CRNN
from readers.featurizer import Wave2Mel

# import sys
# sys.path.append(r'C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID')
ROOT = "F:/DATAS/NEUCOUGHDATA_COUGHSINGLE/"

# Step 1 观察数据集

## 1.1 尝试根据时间截取片段

In [ ]:
slice_raw = pd.read_csv(ROOT+'neucough_metainfo_slice.txt', header=0, index_col=0)
slice_raw

In [ ]:
def min2sec(t: str):
    parts = t.split(':')
    return int(parts[0]) * 60 + float(parts[1])

def read_audio(filepath: str, st=None, en=None):
    # print(y.shape, sr)
    # sr = 22050
    if (st is not None) and (en is not None):
        # st, en = int(st * sr), int(en * sr)
        # print("st, en:", st, en)
        y, sr = librosa.load(filepath, offset=st, duration=en - st)
        # print("y, sr:", len(y), sr)
    else:
        y, sr = librosa.load(filepath, )
        # print("y, sr:", y, sr)
    # print(y.shape)
    # mel = w2m(torch.from_numpy(y))
    return y, sr#  , mel

In [ ]:
for idx, item in enumerate(slice_raw.itertuples()):
    if idx >0:
        break
    name, st, en = item[0], item[1], item[2]
    print(name, st, en)
    audio_path = "F:/DATAS/NEUCOUGHDATA_FULL/{}_audiodata_元音字母a.wav".format(name)
    y = read_audio(audio_path, st=min2sec(st), en=min2sec(en))
    # plt.subplot(5, 1, idx+1)
    plt.figure(idx, figsize=(14, 5))
    plt.plot(y)

## 1.2 统计波形长度直方图

In [ ]:

sns.displot(slice_raw, x="slice", bins=10, height=4, aspect=3)

# Step 2 统计Mel谱图的形状，统一长度，长的Crop短的Padding

## 2.1 读取、按时间切分、转换Mel谱

In [ ]:

class Wave2Mel(object):
    def __init__(self, sr,
                 n_fft=1024,
                 n_mels=128,
                 win_length=1024,
                 hop_length=512,
                 power=2.0
                 ):
        self.mel_transform = torchaudio.transforms.MelSpectrogram(sample_rate=sr,
                                                                  win_length=win_length,
                                                                  hop_length=hop_length,
                                                                  n_fft=n_fft,
                                                                  n_mels=n_mels,
                                                                  power=power)
        self.amplitude_to_db = torchaudio.transforms.AmplitudeToDB(stype='power')

    def __call__(self, x):
        return self.amplitude_to_db(self.mel_transform(x))

In [ ]:
mel_length_list = []
sr_list = []
w2m = Wave2Mel(sr=22050)
fout = open("F:/DATAS/NEUCOUGHDATA_COUGHSINGLE/neucough_slicemel.txt", 'w')
fout.write("filename,duration,slice\n")
for idx, item in tqdm(enumerate(slice_raw.itertuples()), desc="Wav2Mel:"):
    name, st, en = item[0], item[1], item[2]
    audio_path = "F:/DATAS/NEUCOUGHDATA_FULL/{}_audiodata_元音字母a.wav".format(name)
    y, sr = read_audio(audio_path, st=min2sec(st), en=min2sec(en))
    sr_list.append(sr)

    mel = w2m(torch.from_numpy(y))
    dim, length = mel.data.numpy().shape
    mel_length_list.append([name, len(y), length])
    fout.write("{},{},{}\n".format(name, len(y), length))
fout.close()

In [ ]:
mel_raw = pd.read_csv(ROOT+'neucough_slicemel.txt', header=0, index_col=0)
mel_raw

## 2.2 统计Mel谱的长度统计直方图

In [ ]:
mel_raw["slice"].mean()

In [ ]:
sns.displot(mel_raw, x="slice", bins=10, height=4, aspect=3)

## 2.3 统一长度，长的Crop短的Padding

### 当前存在一个问题：有的很长，可以随机截取多个，而不是只要一个

In [ ]:
fixed_length = 16
for idx, item in enumerate(slice_raw.itertuples()):
    name, st, en = item[0], item[1], item[2]
    st, en = min2sec(st), min2sec(en)
    audio_path = "F:/DATAS/NEUCOUGHDATA_FULL/{}_audiodata_元音字母a.wav".format(name)
    y, sr = read_audio(audio_path, st=min2sec(st), en=min2sec(en))
    mel = w2m(torch.from_numpy(y)).data.numpy()
    dim, length = mel.shape
    if length < 16:
        new_mel = np.zeros((128, 16))
        st = np.random.randint(0, (fixed_length-length+1)//2)
        new_mel[:, st:st+16] = mel
    elif length>16:
        st = np.random.randint(0, (length+1-fixed_length)//2)
        new_mel = mel[:, st:st+16]
    else:
        new_mel = mel
        # print("wav, mel length:", len(y), mel.shape)
    print(new_mel.shape)

# Step 3 从原始音频中随机截取一些负样本，作为训练，和Region Proposal

## 寻找切分多大的范围内，波形的mel谱才是16

In [ ]:
def min2sec(t: str):
    parts = t.split(':')
    return int(parts[0]) * 60 + float(parts[1])

def get_rand_start(sec: int, wav_length:int, mode="left", full_length: int=None, sr=22050):
    """ 除法/不要写成//，会导致算什么都是0 """
    if mode=="left":
        return np.random.randint(0, sec*sr-wav_length) / sr
    elif mode=="right":
        return np.random.randint(sec*sr, full_length-wav_length) / sr
    else:
        raise Exception("Unknown mode of get_rand_start(mode=\"???\").")

def read_audio(filepath: str, st=None, en=None, duration=None):
    # print(y.shape, sr)
    # sr = 22050
    if st is not None:
        if en is not None:
            # st, en = int(st * sr), int(en * sr)
            # print("st, en:", st, en)
            y, sr = librosa.load(filepath, offset=st, duration=en - st)
            # print("y, sr:", len(y), sr)
        elif duration is not None:
            y, sr = librosa.load(filepath, offset=st, duration=duration)
    else:
        y, sr = librosa.load(filepath)
            # print("y, sr:", y, sr)
    # print(y.shape)
    # mel = w2m(torch.from_numpy(y))
    return y, sr#  , mel

In [ ]:
def labelling_by_IOU(st1:int, en1:int, st2:int, d2:int, mode="left"):
    print(st1,en1,st2,d2)
    if mode=="left":
        if st2+d2<st1:
            return 0  # other
        elif (st2+d2-st1)/(en1-st2) > 0.2:
            return 1
        else:
            return 0
    else:
        raise Exception("Unknown mode of labelling_by_IOU(mode=\"???\")")

In [ ]:
lab2name = {0: "non-cough", 1:"cough"}
fixed_length = 16
for idx, item in enumerate(slice_raw.itertuples()):
    name, st, en = item[0], item[1], item[2]
    st, en = min2sec(st), min2sec(en)
    print("st, en:", st, en)
    audio_path = "F:/DATAS/NEUCOUGHDATA_FULL/{}_audiodata_元音字母a.wav".format(name)
    pos_sample, sr = read_audio(audio_path, st=st, en=en)
    pos_mel = w2m(torch.from_numpy(pos_sample).to(torch.float32))
    print("pos_sample:", pos_sample.shape, pos_mel.shape)
    print("label:", 1, lab2name[1])
    
    mel16_rand_wavlen = np.random.randint(7680, 8192)  # Mel length: 16
    print("wavlen:", mel16_rand_wavlen/sr)
    rand_start = get_rand_start(sec=st, wav_length=mel16_rand_wavlen, mode="left", full_length=None, sr=22050)
    print("rand_start:", rand_start)
    neg_sample, _ = read_audio(audio_path, st=rand_start, duration=mel16_rand_wavlen/sr)  # 除法/不要写成//，会导致算什么都是0
    neg_mel = w2m(torch.from_numpy(neg_sample).to(torch.float32))
    print("neg_sample:", neg_sample.shape, neg_mel.shape)
    gen_label = labelling_by_IOU(st1=st,en1=en,st2=rand_start,d2=mel16_rand_wavlen/sr)
    print("label:", gen_label, lab2name[gen_label])
    while gen_label==1:
        mel16_rand_wavlen = np.random.randint(7680, 8192)  # Mel length: 16
        print("wavlen:", mel16_rand_wavlen/sr)
        rand_start = get_rand_start(sec=st, wav_length=mel16_rand_wavlen, mode="left", full_length=None, sr=22050)
        print("rand_start:", rand_start)
        neg_sample, _ = read_audio(audio_path, st=rand_start, duration=mel16_rand_wavlen/sr)  # 除法/不要写成//，会导致算什么都是0
        print("neg_sample:", neg_sample.shape)
        gen_label = labelling_by_IOU(st1=st,en1=en,st2=rand_start,d2=mel16_rand_wavlen/sr)
        print("label:", gen_label, lab2name[gen_label])
    print()
    if idx>2:
        print("idx:", idx, mel16_rand_wavlen)
        break

# Step 4 均等分割一条数据

## 4.1 KMeans聚类算法

### RMS能量计算
- 想法：带重叠滑动窗口获取样本，然后画图看一看是否可分
- 然后多个文件一起，线性判别分析

In [ ]:
import os
import numpy as np
import random
import librosa
import matplotlib.pyplot as plt

In [ ]:
def sig2rms(signal):
    return np.sqrt(np.mean(np.square(signal)))

def threshold2x(x, tr, mode="origin"):
    if mode=="origin":
        return x if x > tr else 0
    elif mode=="label":
        return 1 if x > tr else 0
    else:
        raise Exception("Error mode of threshold2x {}, please choise from [\"origin\", \"label\"].".format(mode))

def file2rmslist(filepath):
    fixed_length = 8191  # mel 16
    overlap = 2048
    wav_input, sr = librosa.load(sample_path)
    N = len(wav_input)
    st = 0
    rms_list = []
    while st < N:
        sample = wav_input[st:st+fixed_length-overlap]
        tmp_rms = sig2rms(sample)
        rms_list.append(threshold2x(tmp_rms))
        st += fixed_length
    return rms
fixed_length = 8191  # mel 16
overlap = 2048

In [ ]:
data_dir = "F:/DATAS/NEUCOUGHDATA_FULL/"
sample_path = "F:/DATAS/NEUCOUGHDATA_FULL/20240921104740_audiodata_元音字母a.wav"
file_list = []
for item in os.listdir("F:/DATAS/NEUCOUGHDATA_FULL/"):
    if item[-3:]=="wav":
        file_list.append(item)
len(file_list)

In [ ]:
def threshold2x(x, tr, mode="origin"):
    if mode=="origin":
        return x if x > tr else 0
    elif mode=="label":
        return 1 if x > tr else 0
    else:
        raise Exception("Error mode of threshold2x {}, please choise from [\"origin\", \"label\"].".format(mode))

def scale01(signal):
    rate = 1. / max(signal)
    return rate * np.array(signal)

sample_path = random.choice(file_list)
file_path = data_dir+sample_path
print(sample_path)
def file2signal(file_path):
    wav_input, sr = librosa.load(file_path)
    N = len(wav_input)
    st = 0
    rms_list = []
    tmp_rms = sig2rms(scale01(wav_input))
    tr = np.mean(tmp_rms) / 6.7
    print("tr:", tr)
    while st < N:
        sample = wav_input[st:st+fixed_length-overlap]
        tmp_rms = sig2rms(sample)
        # print(np.mean(tmp_rms))
        rms_list.append(threshold2x(tmp_rms, tr))
        st += fixed_length
    return rms_list
rms_list = file2signal(file_path)
plt.figure(0)
plt.plot(range(len(wav_input)), wav_input, c="black")
plt.figure(1)
plt.stem(range(len(rms_list)), rms_list)
plt.show()

## 4.2 尝试一下线性判别分析
- 因为我需要的分类任务就是最精准的线性分类
- 但是我感觉我不需要分类，我已经手动找阈值了，标签我都是通过阈值赋予的，那我还分什么类啊！！！
- 说起来我的目标，不就是识别静音吗！！！

In [ ]:
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

## 进行Region Proposal的方法之我的想法
- 根据我的调查，从RCNN开始就有的方法，先进行Region Proposal，具体方法，以及输出后的处理我不清楚，我怀疑不同region尺寸不一样大无法组成Batch。
- 关注一下Bounding-Boxes的选取
- YOLO里面提到了Anchor，锚框的选择值得关注
- 我的想法则是最古老的方法，滑动窗口，带重叠。可以先试试。

In [ ]:
wav2mel = Wave2Mel(sr=22050)

In [ ]:
test_len = [8192, 4068, 6144, 7168, 7679, 7680, 7681, 8191, 8192]
for length in test_len:
    test_x = torch.from_numpy(np.random.rand(length)).to(torch.float32)
    x_mel = wav2mel(test_x)
    print(length, x_mel.shape)


In [ ]:
number = 8191
for factor in range(2, int(np.sqrt(number))+2):
    if number % factor == 0:
        print(factor, number/factor)
print("prime")

In [ ]:
sample_path = "F:/DATAS/NEUCOUGHDATA_FULL/20240921104740_audiodata_元音字母a.wav"
# pool_size = [8, 8, 2]
params = {"pool_size": [8, 8, 2], "dropout_rate": 0.0, "batch_size": 32, "nb_cnn2d_filt": 64,
          "rnn_size": [128, 128], "fnn_size": [256, 128, 32]}
device = torch.device("cuda")
model = CRNN(params=params).to(device)
model.load_state_dict(torch.load("./runs/sed_crnn/{}/epoch_{}_sedmodel.pth".format("202411152222", 19)))
model.eval()
print(model)

# sample_list, label_list = read_fullwave_file()
lab2name = {0: "non-cough", 1: "cough"}
fixed_length = 8191  # mel 16
overlap = 2048

In [ ]:
wav_input, sr = librosa.load(sample_path)
N = len(wav_input)
st = 0
st_list = []
yhat_list = []
while st < N:
    sample = wav_input[st:st+fixed_length-overlap]
    x_mel = wav2mel(torch.from_numpy(sample).to(torch.float32))
    x_mel = x_mel.unsqueeze(0).to(device)
    print("start:{}, x_mel.shape:{}".format(st, x_mel.shape))
    with torch.no_grad():
        y_pred, _ = model(x_mel)
        y_hat = torch.argmax(y_pred, dim=-1)
        st_list.append(st)
        yhat_list.append(y_hat)
    st += fixed_length

In [ ]:
yhat_list = [item.data.cpu().numpy() for item in yhat_list]

In [ ]:
plt.figure(1)
plt.stem(range(len(yhat_list)), yhat_list)
plt.show()

# End